# SetFit for Multilabel Text Classification

In this notebook, we'll learn how to do few-shot text classification on a multilabel dataset with SetFit.

## Setup

If you're running this Notebook on Colab or some other cloud platform, you will need to install the `setfit` library. Uncomment the following cell and run it:

In [1]:
%pip install setfit

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Alternatively, if you are running directly from source, you can set the path to the setfit source code

In [2]:
import sys
sys.path.append('../src')

To be able to share your model with the community, there are a few more steps to follow.

First, you have to store your authentication token from the Hugging Face Hub (sign up [here](https://huggingface.co/join) if you haven't already!). To do so, execute the following cell and input an [access token](https://huggingface.co/docs/hub/security-tokens) associated with your account:

Then you need to install Git-LFS, which you can do by uncommenting and running following command:

In [3]:
# !apt install git-lfs

This notebook is designed to work with any multiclass [text classification dataset](https://huggingface.co/models?pipeline_tag=text-classification&sort=downloads) and pretrained [Sentence Transformer](https://huggingface.co/models?library=sentence-transformers&sort=downloads) on the Hub. Change the values below to try a different dataset / model!

In [4]:
from datasets import load_dataset

model_id = "sentence-transformers/paraphrase-mpnet-base-v2"
dataset = load_dataset("ethos", "multilabel")

  0%|          | 0/1 [00:00<?, ?it/s]

In [5]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'violence', 'directed_vs_generalized', 'gender', 'race', 'national_origin', 'disability', 'religion', 'sexual_orientation'],
        num_rows: 433
    })
})

## Loading and sampling the dataset

In [6]:
dataset.data

{'train': MemoryMappedTable
 text: string
 violence: int64
 directed_vs_generalized: int64
 gender: int64
 race: int64
 national_origin: int64
 disability: int64
 religion: int64
 sexual_orientation: int64
 ----
 text: [["You should know women's sports are a joke","You look like Sloth with deeper Down’s syndrome","You look like Russian and speak like Indian. Both are disgusting go kill yourself","Women deserve to be abused, I guess.","Women are made for making babies and cooking dinner and nothing else!!!","why haven't you faggots taken the bi-pill yet","Why don’t you go beck your god to murder you? Christianity is a joke","why does that african american monkey hold gun like that?","Why does Bulgaria exist as a country? It's one of those homosexual ones.","Why do negroes tear down their own neighborhoods when a black thug gets killed by law enforcement? Must be because of low IQ.",...,"Come and kill my wife","Catholicism is the beast that comes out of daniels dream interpretation.","By

Most datasets on the Hub have many more labeled examples than those one encounters in few-shot settings. To simulate the effect of training on a limited number of examples, let's subsample the training set to have at least 8 labeled examples per feature.

Note that if your dataset has differently formatted labels, you may need to adapt this section.

In [7]:
import numpy as np

features = dataset["train"].column_names
features.remove("text")
features

['violence',
 'directed_vs_generalized',
 'gender',
 'race',
 'national_origin',
 'disability',
 'religion',
 'sexual_orientation']

In [8]:
num_samples = 8
samples = np.concatenate(
    [np.random.choice(np.where(dataset["train"][f])[0], num_samples) for f in features]
)

In [9]:
samples

array([288,  82, 145, 189, 392, 407, 372, 189, 318, 160,  97, 369, 421,
       353, 291, 275, 420, 333, 365, 189, 190,   0, 251,   3,  48, 118,
       426, 426, 121, 335,   7, 174, 380, 314, 244, 387,  27, 329, 329,
        27, 361, 150, 172,  28, 332,  98, 258,  73,  75, 138, 255, 141,
       261, 374,  80,  15, 176, 124,   8, 365, 371, 128,  36, 275])

We encode the emotions in a single `'label'` feature. 

In [10]:
def encode_labels(record):
    return {"labels": [record[feature] for feature in features]}


dataset = dataset.map(encode_labels)

  0%|          | 0/433 [00:00<?, ?ex/s]

In [11]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'violence', 'directed_vs_generalized', 'gender', 'race', 'national_origin', 'disability', 'religion', 'sexual_orientation', 'labels'],
        num_rows: 433
    })
})

In [12]:
import pandas as pd

df = pd.DataFrame.from_records(dataset['train'])
print(df.shape)
df

(433, 10)


,text,violence,directed_vs_generalized,gender,race,national_origin,disability,religion,sexual_orientation,labels
0,You should know women's sports are a joke,0,0,1,0,0,0,0,0,"[0, 0, 1, 0, 0, 0, 0, 0]"
1,You look like Sloth with deeper Down’s syndrome,0,1,0,0,0,1,0,0,"[0, 1, 0, 0, 0, 1, 0, 0]"
2,You look like Russian and speak like Indian. B...,1,1,0,0,1,0,0,0,"[1, 1, 0, 0, 1, 0, 0, 0]"
3,"Women deserve to be abused, I guess.",1,0,1,0,0,0,0,0,"[1, 0, 1, 0, 0, 0, 0, 0]"
4,Women are made for making babies and cooking d...,0,0,1,0,0,0,0,0,"[0, 0, 1, 0, 0, 0, 0, 0]"
...,...,...,...,...,...,...,...,...,...,...
428,All people are fucked up because of these quee...,0,0,1,0,0,0,0,1,"[0, 0, 1, 0, 0, 0, 0, 1]"
429,Actually THIS WEIRD PEOPLE are HARMFUL! They n...,0,1,0,0,0,1,0,0,"[0, 1, 0, 0, 0, 1, 0, 0]"
430,ABU DA BI ABA DU BA BA HAMUD! See? I’m an arab...,0,0,0,0,0,0,1,0,"[0, 0, 0, 0, 0, 0, 1, 0]"
431,a homo? so you fuck your brother instead of yo...,0,1,0,0,0,0,0,1,"[0, 1, 0, 0, 0, 0, 0, 1]"


Next, we use the samples we selected as our training set, and the others as our test set (since the ethos dataset does not have a test split on the hub).

Here we have 64 total examples to train with since the `ethos` dataset has 8 classes.

In [13]:
train_dataset = dataset["train"].select(samples)
eval_dataset = dataset["train"].select(
    np.setdiff1d(np.arange(len(dataset["train"])), samples)
)

Okay, now we have the dataset, let's load and train a model!

## Fine-tuning the model

To train a SetFit model, the first thing to do is download a pretrained checkpoint from the Hub. We can do so by using the `from_pretrained()` method associated with the `SetFitModel` class.

**Note that the `multi_target_strategy` parameter here signals to both the model and the trainer to expect a multi-labelled dataset.**

In [14]:
from setfit import SetFitModel

model = SetFitModel.from_pretrained(model_id, multi_target_strategy="one-vs-rest")

model_head.pkl not found on HuggingFace Hub, initialising classification head with random weights. You should TRAIN this model on a downstream task to use it for predictions and inference.


Here, we've downloaded a pretrained Sentence Transformer from the Hub and added a logistic classification head to the create the SetFit model. As indicated in the message, we need to train this model on some labeled examples. We can do so by using the `SetFitTrainer` class as follows:

In [15]:
from sentence_transformers.losses import CosineSimilarityLoss
from setfit import SetFitTrainer

trainer = SetFitTrainer(
    model=model,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    loss_class=CosineSimilarityLoss,
    batch_size=8,
    num_epochs=1,
    num_iterations=20,
    column_mapping={"text": "text", "labels": "label"},
)

The main arguments to notice in the trainer is the following:

* `loss_class`: The loss function to use for contrastive learning with the Sentence Transformer body
* `num_iterations`: The number of text pairs to generate for contrastive learning
* `column_mapping`: The `SetFitTrainer` expects the inputs to be found in a `text` and `label` column. This mapping automatically formats the training and evaluation datasets for us.

Now that we've created a trainer, we can train it!

In [ ]:
trainer.train()

Applying column mapping to training dataset
***** Running training *****
  Num examples = 3780
  Num epochs = 1
  Total optimization steps = 473
  Total train batch size = 8


Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Iteration:   0%|          | 0/473 [00:00<?, ?it/s]

The final step is to compute the model's performance using the `evaluate()` method. The default metric measures 'subset accuracy', which measures the fraction of samples where we predict all 8 labels correctly.

In [ ]:
metrics = trainer.evaluate()
metrics

And once the model is trained, you can push it to the Hub:

In [ ]:
trainer.push_to_hub(f"setfit-ethos-multilabel-example")

You can now share this model with all your friends, family, favorite pets: they can all load it with the identifier `your-username/the-name-you-picked` so for instance:

In [ ]:
from setfit import SetFitModel
model = SetFitModel.from_pretrained("lewtun/setfit-ethos-multilabel-example")

Run inference. As is usual in toxicity models, it tends to think any mention of topics such as race or gender are negative.

In [ ]:
preds = model(
    [
        "Jewish people often don't eat pork.",
        "Is this lipstick suitable for people with dark skin?"
    ]
)
preds

In [ ]:
# Show predicted labels, requires you to have stored the 'features' somewhere
[[f for f, p in zip(features, ps) if p] for ps in preds]